# Литература
[Practitioner’s Guide to Statistical Tests](https://vkteam.medium.com/practitioners-guide-to-statistical-tests-ed2d580ef04f#d2d3)

[А/Б тесты с метрикой отношения. Дельта-метод](https://habr.com/ru/companies/X5Tech/articles/740476/)

[A/B-тесты с метриками-отношениями и при чём здесь внутрипользовательские корреляции](https://www.youtube.com/watch?v=ObzlKVCiBqI&list=PLAynZk3jFRd_81lZ1W0E9XEyXtAljkBlh&index=7&t=5201s)